In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


#### Prepare Stopwords

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

/home/sandeep/anaconda3/lib/python3.8/site-packages/nltk/decorators.py:67: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  signature = inspect.formatargspec(
/home/sandeep/anaconda3/lib/python3.8/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Sequence, defaultdict


#### Import Newsgroups Data

In [3]:
# Import Dataset
df = pd.read_json('/home/sandeep/Desktop/samudsan/Machine learning/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


#### Remove emails and newline characters

In [4]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub(r'S*@S*s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub(r's+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]

pprint(data[:1])

['From: lerx twam.umd.edu (where  my thing)\n'
 'Subject: WHAT car i  thi !?\n'
 'Nntp-Po ting-Ho t: rac3.wam.umd.edu\n'
 'Organization: Univer ity of Maryland, College Park\n'
 'Line : 15\n'
 '\n'
 ' I wa  wondering if anyone out there could enlighten me on thi  car I  aw\n'
 'the other day. It wa  a 2-door  port  car, looked to be from the late 60 /\n'
 'early 70 . It wa  called a Bricklin. The door  were really  mall. In '
 'addition,\n'
 'the front bumper wa   eparate from the re t of the body. Thi  i  \n'
 'all I know. If anyone can tellme a model name, engine  pec , year \n'
 'of production, where thi  car i  made, hi tory, or whatever info you\n'
 'have on thi  funky looking car, plea e e-mail.\n'
 '\n'
 'Thank ,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerx t ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


####  Tokenize words and Clean-up text

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['from', 'lerx', 'twam', 'umd', 'edu', 'where', 'my', 'thing', 'subject', 'what', 'car', 'thi', 'nntp', 'po', 'ting', 'ho', 'rac', 'wam', 'umd', 'edu', 'organization', 'univer', 'ity', 'of', 'maryland', 'college', 'park', 'line', 'wa', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'thi', 'car', 'aw', 'the', 'other', 'day', 'it', 'wa', 'door', 'port', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'wa', 'called', 'bricklin', 'the', 'door', 'were', 'really', 'mall', 'in', 'addition', 'the', 'front', 'bumper', 'wa', 'eparate', 'from', 'the', 're', 'of', 'the', 'body', 'thi', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'pec', 'year', 'of', 'production', 'where', 'thi', 'car', 'made', 'hi', 'tory', 'or', 'whatever', 'info', 'you', 'have', 'on', 'thi', 'funky', 'looking', 'car', 'plea', 'mail', 'thank', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerx']]


#### Creating Bigram and Trigram Models

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'lerx', 'twam', 'umd', 'edu', 'where', 'my', 'thing', 'subject', 'what', 'car', 'thi', 'nntp', 'po', 'ting_ho', 'rac_wam_umd', 'edu', 'organization', 'univer_ity', 'of', 'maryland_college_park', 'line', 'wa', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'thi', 'car', 'aw', 'the', 'other', 'day', 'it', 'wa', 'door', 'port', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'wa', 'called', 'bricklin', 'the', 'door', 'were', 'really', 'mall', 'in', 'addition', 'the', 'front_bumper', 'wa', 'eparate', 'from', 'the', 're', 'of', 'the', 'body', 'thi', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'pec', 'year', 'of', 'production', 'where', 'thi', 'car', 'made', 'hi_tory', 'or', 'whatever', 'info', 'you', 'have', 'on', 'thi', 'funky', 'looking', 'car', 'plea', 'mail', 'thank', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerx']


#### Remove Stopwords, Make Bigrams and Lemmatize


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['thi', 'nntp', 'park', 'line', 'wonder', 'could', 'enlighten', 'thi', 'car', 'day', 'door', 'port', 'car', 'look', 'late', 'early', 'call', 'really', 'mall', 'addition', 'know', 'model', 'name', 'year', 'production', 'thi', 'car', 'make', 'info', 'thi', 'funky', 'look', 'car', 'plea', 'mail', 'thank', 'bring', 'neighborhood', 'lerx']]


#### Create the Dictionary and Corpus needed for Topic Modeling

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.



In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


# Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

[[(0, 1), (1, 1), (2, 1), (3, 4), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1)]]


In [9]:
id2word[0]

'addition'

In [10]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 4),
  ('could', 1),
  ('day', 1),
  ('door', 1),
  ('early', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerx', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('mall', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp', 1),
  ('park', 1),
  ('plea', 1),
  ('port', 1),
  ('production', 1),
  ('really', 1),
  ('thank', 1),
  ('thi', 4),
  ('wonder', 1),
  ('year', 1)]]

### Building the Topic Model

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#### View the topics in LDA model

In [12]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.044*"people" + 0.023*"think" + 0.021*"come" + 0.020*"right" + 0.019*"go" '
  '+ 0.018*"aid" + 0.018*"take" + 0.015*"even" + 0.014*"make" + 0.013*"man"'),
 (1,
  '0.053*"year" + 0.027*"go" + 0.026*"car" + 0.022*"high" + 0.019*"next" + '
  '0.018*"run" + 0.016*"low" + 0.015*"old" + 0.015*"will" + 0.015*"win"'),
 (2,
  '0.092*"terrori" + 0.067*"internal" + 0.051*"reque" + 0.046*"letter" + '
  '0.044*"procedure" + 0.040*"cable" + 0.040*"upgrade" + 0.034*"clock" + '
  '0.033*"chri_tianity" + 0.031*"economic"'),
 (3,
  '0.033*"may" + 0.032*"evidence" + 0.025*"point" + 0.023*"claim" + '
  '0.015*"however" + 0.015*"many" + 0.015*"believe" + 0.014*"true" + '
  '0.014*"example" + 0.014*"mean"'),
 (4,
  '0.062*"power" + 0.059*"type" + 0.034*"bike" + 0.033*"light" + '
  '0.026*"monitor" + 0.022*"develop" + 0.022*"ride" + 0.022*"peed" + '
  '0.020*"science" + 0.020*"advice"'),
 (5,
  '0.079*"faith" + 0.069*"book" + 0.063*"religion" + 0.044*"truth" + '
  '0.042*"belief" + 0.035*"cience" + 

#### Compute Model Perplexity and Coherence Score

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.824847169458021

Coherence Score:  0.457559041198892


#### Visualize the topics-keywords

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/sandeep/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.336369 -0.132215       1        1  30.220743
0      0.305379 -0.139573       2        1  18.282776
3      0.229378 -0.144289       3        1   8.775469
1      0.191341  0.053211       4        1   8.022851
10     0.109010  0.331662       5        1   7.670540
9      0.196355 -0.005973       6        1   7.134679
6      0.053046  0.181810       7        1   3.969160
8      0.073777  0.024626       8        1   3.356929
4     -0.037151  0.090840       9        1   2.388725
19    -0.010436  0.107342      10        1   2.336683
5     -0.072825  0.027388      11        1   1.504276
13    -0.070781  0.029929      12        1   1.408271
18    -0.066292 -0.035441      13        1   1.178024
14    -0.157934 -0.047435      14        1   0.749869
12    -0.166260 -0.043714      15        1   0.670866
16    -0.162501 -0.055038      16        1   0.603309
17    -0.168224 -0.052410      17        1   0.588234
11    -0.182595 -0.059627      18        1   0.496265
2     -0.181908 -0.059315      19        1   0.486405
15    -0.217748 -0.071777      20        1   0.155922, topic_info=      Category          Freq     Term         Total  loglift  logprob
14     Default  14865.000000     line  14865.000000  30.0000  30.0000
174    Default  10920.000000    write  10920.000000  29.0000  29.0000
157    Default   9590.000000      com   9590.000000  28.0000  28.0000
154    Default   8615.000000  article   8615.000000  27.0000  27.0000
151    Default  16857.000000    would  16857.000000  26.0000  26.0000
...        ...           ...      ...           ...      ...      ...
7423   Topic20      0.029852    slave      1.209968   2.7615 -10.8354
650    Topic20      0.029852  leather      1.209856   2.7616 -10.8354
2932   Topic20      0.029852      fun      1.209876   2.7615 -10.8354
10045  Topic20      0.029852   person      1.209871   2.7615 -10.8354
5560   Topic20      0.029852      ink      1.209895   2.7615 -10.8354

[862 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
700       1  0.999117          able
65        7  0.272266          acce
65        8  0.451350          acce
65       10  0.275178          acce
739      16  0.968957  acceleration
...     ...       ...           ...
151       1  0.896113         would
151       2  0.103872         would
1689     20  0.925121         wound
174       5  0.999862         write
31        4  0.999694          year

[1030 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 4, 2, 11, 10, 7, 9, 5, 20, 6, 14, 19, 15, 13, 17, 18, 12, 3, 16])

ref: 
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
    